## Load libraries

In [ ]:
import json
import re
from redis import Redis
from mstr_robotics.json_compare import Export_Dpn_Obj, JSONDiffAnalyzer,read_and_compare_obj_def
import pandas as pd

from mstrio.connection import Connection

def run_json_diff_analysis( original_folder: str, comparison_folder: str) -> pd.DataFrame:
    try:
        # Initialize with both folders
        analyzer = JSONDiffAnalyzer(original_folder, comparison_folder)

        # Compare matching files between folders
        results_df = analyzer.analyze_directory(comparison_strategy="cross_folder")

        analyzer.save_results("D:/shared_drive/Python/mstr_robotics/json_compare/json_diff_analysis.csv", format="csv")
        
        out_d={"results_df":results_df,"fin_df":pd.read_csv("D:/shared_drive/Python/mstr_robotics/json_compare/json_diff_analysis.csv")}
        return out_d

    except Exception as e:
        print(f"Error during analysis: {e}")


ModuleNotFoundError: No module named 'mstr_robotics.read_and_compare_obj_def'

## Connect to MSTR

In [ ]:
with open('..\\config\\user_d.json', 'r') as openfile:
    user_d = json.load(openfile)
conn_params =  user_d["conn_params"]
conn = Connection(**conn_params)

## Define compare

In [ ]:
play_book_d = {
    "control_dpn_d": {"recurs_fg": "true", "all_schema_obj_fg": "false", "true_mstr_obj_l": ["1","2","3","4","7", "10","12","13", 
                            "14","43", "47", "55"]},
    "keys_to_remove": {"dateModified", "dateCreated"},
    "compare_d": [
        {
            "conn_params": {
                "username": "Administrator",
                "password": "Victoria_01",
                "base_url": "http://85.214.60.83:8080/MicroStrategyLibrary/api"
            },
            "project_id": "B7CA92F04B9FAE8D941C3E9B7E0CD754",
            "obj_id": "98EB31B54122FFB738E6E08A2F29421A",
            "obj_type": "55",
            "subtype": "14081",
            "out_dir": "D:/shared_drive/Python/mstr_robotics/json_compare/system1_json"
        },
        {
            "conn_params": {
                "username": "Administrator",
                "password": "Victoria_01",
                "base_url": "http://85.214.60.83:8080/MicroStrategyLibrary/api"
            },
            "project_id": "00F3103E41FF3743A057C3B5313595B7",
            "obj_id": "98EB31B54122FFB738E6E08A2F29421A",
            "obj_type": "55",
            "subtype": "14081",
            "out_dir": "D:/shared_drive/Python/mstr_robotics/json_compare/system2_json"
        }
    ]
}

# Create an instance of the class and run the analysis
analyzer = Export_Dpn_Obj()
analyzer.read_save_compare_obj(play_book_d)

original_folder = "D:/shared_drive/Python/mstr_robotics/json_compare/system1_json"
comparison_folder = "D:/shared_drive/Python/mstr_robotics/json_compare/system2_json"
output_folder = "D:/shared_drive/Python/mstr_robotics/json_compare"

out_d=run_json_diff_analysis(original_folder=original_folder, comparison_folder=comparison_folder)


In [ ]:
def search_obj_dpn_tree(conn, obj_id, recurs_fg="true"):
    url = f'{conn.base_url}/api/metadataSearches/results?pattern=4&domain=2&scope=standalone&usedByObject={obj_id}&usedByRecursive={recurs_fg}&visibility=ALL'

    search_inst_resp = conn.post(url)
    if search_inst_resp.status_code == 200:
        search_id=search_inst_resp.json()["id"]
        url = f'{conn.base_url}/api/metadataSearches/results/tree?searchId={search_id}&offset=0&limit=-1' 
        search_resp = conn.get(url)
        if search_resp.status_code == 200:
            obj_d_l=search_resp.json()
            return obj_d_l
conn.select_project(project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754")
json_file=search_obj_dpn_tree(conn, obj_id="98EB31B54122FFB738E6E08A2F29421A;55", recurs_fg="true")

with open('D:/shared_drive/Python/mstr_robotics/json_compare/dpn_tree.json', 'w') as f:
    json.dump(json_file, f, indent=4)   

    


## upload files to redis

In [ ]:
# Connect to your Redis Cloud instance
r = Redis(
    host='redis-14995.c56.east-us.azure.redns.redis-cloud.com',
    port=14995,
    password='KSvQA45EE8ZEJ3xcFvus96927CW56D8u',  # Redis Cloud requires auth
    username='default',  # Redis Cloud requires username
    decode_responses=True
)

# Batch upload with pipeline
def bulk_upload_json_files(folder_path, batch_size=100):
    files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
    
    for i in range(0, len(files), batch_size):
        batch = files[i:i + batch_size]
        pipe = r.pipeline()
        try:
            for filename in batch:
                
                filepath = os.path.join(folder_path, filename)
                with open(filepath, 'r') as f:
                    data = json.load(f)
                
                # Store as JSON (Redis 6.2+) or serialized string
                key = f"file:{filename.replace('.json', '')}"
                pipe.json().set(key, '$', data)  # or pipe.set(key, json.dumps(data))
            
            pipe.execute()
            print(f"Uploaded batch {i//batch_size + 1}/{(len(files) + batch_size - 1)//batch_size}")
        except Exception as e:
            print(f"Error uploading batch {i//batch_size + 1}: {e}")
bulk_upload_json_files('D:/shared_drive/Python/mstr_robotics/mstr_all_proj_obj_def', batch_size=100)